# ResST Tutorial
Author: Jinjin Huang


## Outline
1. Installation
2. Import modules
3. Reading in data
4. Enhancing gene expression matrix
5. Training model
6. Clustering
7. Visualization

## 1. Installation
To install ResST package you must make sure that your python version is over 3.7.=. If you don’t know the version of python you can check it by:

In [1]:
import platform
platform.python_version()

'3.7.0'

Note: Because ResST pends on pytorch, you should make sure torch is correctly installed.

Download the package from Github and install it locally:

In [2]:
git clone https://github.com/StickTaTa/ResST_main.git
cd ResST_main
pip install -r requirements.txt

## 2. Import modules

In [3]:
import anndata
import pandas as pd
import scanpy as sc
import os

import sklearn
from scipy.spatial import distance

from resst.model_ST_utils import trainer, priori_cluster, plot_map
from resst.get_adata import get_data, refine
from resst.preprocess import get_enhance_feature

## 3. Reading in data
Data can be downloaded from [Human dorsolateral prefrontal cortex 10× Visium data](http://research.libd.org/spatialLIBD/), [Mouse brain 10× Visium data](https://support.10xgenomics.com/spatial-gene-expression/datasets), [Human breast cancer 10× Visium data](https://support.10xgenomics.com/spatial-gene-expression/datasets), [4i and MIBI-TOF dataset](https://github.com/scverse/squidpy), [mouse hippocampus dataset](https://portals.broadinstitute.org/single_cell/study/slide-seq-study), [Hypothalamic Preoptic Region MERFISH dataset](https://doi.org/10.1126%2Fscience.aau5324), [Adult Macaque Cortex Stereoseq dataset](https://www.nature.com/articles/s41467-022-34413-3), [Sagittal mouse embryos seqFISH dataset](https://www.nature.com/articles/s41587-021-01006-2).


In [4]:
# setting data_name, generated_data_path, model_path(save model), result_path(save results)
data_name = 'V1_Breast_Cancer_Block_A_Section_1'
generated_data_path = 'generated_data'
model_path = os.path.join('model/', data_name)
result_path = os.path.join('results/', data_name)

if not os.path.exists(generated_data_path):
    os.makedirs(generated_data_path)
if not os.path.exists(model_path):
    os.makedirs(model_path)
if not os.path.exists(result_path):
    os.makedirs(result_path)

In [5]:
adata = get_data(data_path='dataset', data_name=data_name,
                 generated_data_path=generated_data_path,
                 cnnType='ResNet50')

C:\Users\18456\anaconda3\envs\tensorflow\lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
Tiling image: 100%|██████████ [ time left: 00:00 ]
Extract image feature: 100%|██████████ [ time left: 00:00 ]  


## 4. Enhancing gene expression matrix

In [6]:
adata = get_enhance_feature(adata)

Physical distance calculating Done!
The number of nearest tie neighbors in physical distance is: 32.199578725645075
Gene correlation calculting Done!
Morphological similarity calculting Done!
The weight result of image feature is added to adata.obsm['weights_matrix_all'] !


Find adjacent spots of each spot: 100%|██████████ [ time left: 00:00 ]


## 5. Training model

In [7]:
adata = trainer(adata, save_path=result_path, data_name=data_name)

Model is training...: 100%|██████████ [ time left: 00:00 ]

resst training has been Done! the embeddings has been stored adata.obsm["embed"].


## 6. Clustering

In [8]:
n_clusters = 19
cluster_adata = anndata.AnnData(adata.obsm["embed"])
cluster_adata.obs_names = adata.obs_names
sc.pp.neighbors(cluster_adata, n_neighbors=15)

res = priori_cluster(cluster_adata, n_domains=n_clusters)

sc.tl.leiden(cluster_adata, key_added="pred", resolution=res, random_state=0)
adata.obs['pred'] = cluster_adata.obs['pred']
######### Strengthen the distribution of points in the model
adj_2d = distance.cdist(adata.obsm['spatial'], adata.obsm['spatial'], 'euclidean')
refined_pred = refine(sample_id=adata.obs.index.tolist(),
                      pred=adata.obs["pred"].tolist(), dis=adj_2d, shape="hexagon")
adata.obs["refine_pred"] = refined_pred

Current count: 36
Current count: 36
Current count: 36
Current count: 36
Current count: 36
Current count: 36
Current count: 36
Current count: 36
Current count: 36
Current count: 35
Current count: 35
Current count: 36
Current count: 37
Current count: 37
Current count: 37
Current count: 34
Current count: 35
Current count: 36
Current count: 35
Current count: 35
Current count: 35
Current count: 35
Current count: 35
Current count: 35
Current count: 33
Current count: 34
Current count: 34
Current count: 34
Current count: 32
Current count: 34
Current count: 34
Current count: 34
Current count: 33
Current count: 33
Current count: 33
Current count: 33
Current count: 33
Current count: 31
Current count: 34
Current count: 34
Current count: 34
Current count: 33
Current count: 33
Current count: 32
Current count: 32
Current count: 30
Current count: 33
Current count: 31
Current count: 31
Current count: 32
Current count: 32
Current count: 30
Current count: 30
Current count: 30
Current count: 30
Current co

## 7. Visualization

In [ ]:
plot_map(adata, save_path=result_path, data_name=data_name)

![本地路径](results/V1_Breast_Cancer_Block_A_Section_1/Figure/V1_Breast_Cancer_Block_A_Section_1/V1_Breast_Cancer_Block_A_Section_1_umap.png)